# DPKE_NTRU_HPS2048509

Esse trabalho foi desenvolvido com base no documento "NTRU Algorithm Specifications And Supporting Documentation".

In [493]:
import random
import unittest

 criar a base de polinômios; $\mathbb{Z}$ é o polinômio de coeficientes inteiros, de forma que $\mathbb{Z}/2\mathbb{Z}$, $\mathbb{Z}/3\mathbb{Z}$, $\mathbb{Z}/q\mathbb{Z}$, são inteiros módulo 2, 3 e q respectivamente.

In [494]:
q=2048
Z.<xZ> = ZZ[]              # ZZ[x]
Z2.<xZ2> = Integers(2)[]   # (ZZ/2ZZ)[x]
Z3.<xZ3> = Integers(3)[]   # (ZZ/3ZZ)[x]
Zq.<xZq> = Integers(q)[]   # (ZZ/2048ZZ)[x]
# xZ = Z.gen()
# xZ.content()
print( xZ.parent())
print(xZ2.parent())
print(xZ3.parent())
print(xZq.parent())

Univariate Polynomial Ring in xZ over Integer Ring
Univariate Polynomial Ring in xZ2 over Ring of integers modulo 2 (using GF2X)
Univariate Polynomial Ring in xZ3 over Ring of integers modulo 3
Univariate Polynomial Ring in xZq over Ring of integers modulo 2048


confirmada a base do anel, implementamos a convolução cíclica, utilizando funções nativas do SageMath

In [495]:
n= 509
Phi_n = lambda x : (x^n-1)//(x-1)
Phi_1 = lambda x : (x - 1)
R.<xR> = Z.quotient(Phi_1(xZ) * Phi_n(xZ))      # ZZ[x] / (x^n-1)
S.<xS> = Z.quotient(Phi_n(xZ))                  # ZZ[x] / (x^(n-1) + x^(n-2) + ... + x + 1)
R3.<xR3> = Z3.quotient(Phi_1(xZ3) * Phi_n(xZ3)) # ZZ[x] / (3, x^n-1)
Rq.<xRq> = Zq.quotient(Phi_1(xZq) * Phi_n(xZq)) # ZZ[x] / (q, x^n-1)              
S2.<xS2> = Z2.quotient(Phi_n(xZ2))              # ZZ[x] / (2, x^(n-1) + x^(n-2) + ... + x + 1)
S3.<xS3> = Z3.quotient(Phi_n(xZ3))              # ZZ[x] / (3, x^(n-1) + x^(n-2) + ... + x + 1)
Sq.<xSq> = Zq.quotient(Phi_n(xZq))              # ZZ[x] / (q, x^(n-1) + x^(n-2) + ... + x + 1)
# R1.<xR1> = Zq.quotient(Phi_1(xZq))

print("R:", xR.parent())
print("S:", xS.parent())
print("R3:", xR3.parent())
print("Rq:", xRq.parent()) 
print("S2:", xS2.parent())
print("S3:", xS3.parent())
print("Sq:", xSq.parent())

R: Univariate Quotient Polynomial Ring in xR over Integer Ring with modulus xZ^509 - 1
S: Univariate Quotient Polynomial Ring in xS over Integer Ring with modulus xZ^508 + xZ^507 + xZ^506 + xZ^505 + xZ^504 + xZ^503 + xZ^502 + xZ^501 + xZ^500 + xZ^499 + xZ^498 + xZ^497 + xZ^496 + xZ^495 + xZ^494 + xZ^493 + xZ^492 + xZ^491 + xZ^490 + xZ^489 + xZ^488 + xZ^487 + xZ^486 + xZ^485 + xZ^484 + xZ^483 + xZ^482 + xZ^481 + xZ^480 + xZ^479 + xZ^478 + xZ^477 + xZ^476 + xZ^475 + xZ^474 + xZ^473 + xZ^472 + xZ^471 + xZ^470 + xZ^469 + xZ^468 + xZ^467 + xZ^466 + xZ^465 + xZ^464 + xZ^463 + xZ^462 + xZ^461 + xZ^460 + xZ^459 + xZ^458 + xZ^457 + xZ^456 + xZ^455 + xZ^454 + xZ^453 + xZ^452 + xZ^451 + xZ^450 + xZ^449 + xZ^448 + xZ^447 + xZ^446 + xZ^445 + xZ^444 + xZ^443 + xZ^442 + xZ^441 + xZ^440 + xZ^439 + xZ^438 + xZ^437 + xZ^436 + xZ^435 + xZ^434 + xZ^433 + xZ^432 + xZ^431 + xZ^430 + xZ^429 + xZ^428 + xZ^427 + xZ^426 + xZ^425 + xZ^424 + xZ^423 + xZ^422 + xZ^421 + xZ^420 + xZ^419 + xZ^418 + xZ^417 + xZ^416 + 

O resultado comprova que tanto o grau quanto as reduções de cada anel estão corretos. Entretanto de acordo com o artigo $Rq$, $Sq$ e $S3$ possuem coeficientes centralizados entre $- q/2$ até $q/2-1$,  assim o resultado até aqui garante que ainda não foi obtida a representação correta.

#### Representantes canônicos

O Sage por padrão gera representantes canônicos com coeficientes dentro de intervalo não negativo $[0, q-1]$, não há uma função nativa que possa mudar esse comportamento. Os representantes canônicos definidos no artigo são específicos para o NTRU. Dessa forma foi implementada a função canonicalNTRU para converter coeficientes para o intervalo centrado.
$$
\begin{align*}
    \underline{R_q} \quad & \text{com grau } n-1 \text{ coeficientes dentro do intervalo } \left[ -q/2, q/2-1\right]. \\
    \underline{S_q} \quad & \text{com grau } n-2 \text{ coeficientes dentro do intervalo } \left[-q/2, q/2-1\right]. \\
    \underline{S3} \quad & \text{com grau } n-2 \text{ coeficientes em } \{-1, 0, 1\}.
\end{align*}
$$

In [496]:
def canonicalNTRU(ring, coeffs, n, q):
    a = ring(coeffs)

    if ring == S3:
        # n = n
        coeffs_can = twos_complement_S3(a, n)

    if ring == Rq:
        # n = n-1
        coeffs_can = twos_complement_coeffs(a, n, q)
    
    else:
        # n = n-1
        coeffs_can = twos_complement_coeffs(a, n, q)
    Z.<xZ> = ZZ[] # novo anel que o elemento pertencerá

    return Z(coeffs_can)

def twos_complement_S3(a, n):
    coeffs = [0] * n

    for i, ai in enumerate(a):
        coeffs[i % n] += int(ai)

    def rep_S3(c):
        r = c % 3
        if r == 2:
            return -1
        return r

    return [rep_S3(c) for c in coeffs]

def twos_complement_coeffs(a, n, q):
    coeffs = [0]*n
    for i, ai in enumerate(a):
        if i < n:
            coeffs[i] += int(ai)

    def rep_centered(val):
        r = int(val) % q
        if r > q//2 or (q % 2 == 0 and r == q//2):
            r = r - q
        return r

    return [rep_centered(c) for c in coeffs]
    

O trecho abaixo busca validar a função canonicalNTRU. Também busca elucidar como gerar polinômios através do sistema algébrico computacional do SageMath. Assim obtemos o representante canônico de acordo com as especificações.

 Apesar de assumirmos que estamos lidando com o representante canônico, o SageMath entende que o polinômio retornado pela função canonical não pertence mais ao anel do polinômio de entrada da função. Para que fosse possível manter a coerção seria necessário inicializar todas os polinômios em pase $\mathbb{Z}[\mathbf{x}]$ sem redução, realizando as reduções "manualmente", entretanto também seria necessário uma função para substituir e manter a eficiencia próxima do integer[].

In [497]:
# criar polinômio no sage a partir de um vetor de coeficientes
P.<x> = ZZ['x'] # todo polinômio deve pertencer a um Anel de Polinômios
coeffs = [-2049, -1025, 1024, 2050]
p = P(coeffs)
print("p: ", P(coeffs))
print ("p em Sq, caso de 0 até q-1: ", Sq(p.list()))
print("p canonical representative: ", canonicalNTRU(Sq, p.list(), n, q))

p:  2050*x^3 + 1024*x^2 - 1025*x - 2049
p em Sq, caso de 0 até q-1:  2*xSq^3 + 1024*xSq^2 + 1023*xSq + 2047
p canonical representative:  2*xZ^3 - 1024*xZ^2 + 1023*xZ - 1


Dessa forma para evitar erros de coersão entre anéis diferentes, utilizaremos uma função que passa o polinômio para base $\mathbb{Z}$, isso equivale a Z(a.list())

In [498]:
def zz(a):
    Z.<xZ> = ZZ['x']
    poly = Z(a.list())
    return poly

In [499]:
coeffs = [1, 2, 4, 5]
test = Rq(coeffs)

result = zz(test)
print(result)

5*x^3 + 4*x^2 + 2*x + 1


#### Sample $\mathcal{T}$ and $\mathcal{T(d)}$

Dado $g$ amostrado de $\mathcal{L}_g=\mathcal{T}$($d$), dessa forma definimos $d=\frac{q}{8}-2$
Onde $0 < d \leq \frac{2n}{3}$. Nesse contexto $d/2$ será o valor que define a quantidade de 1 e -1, que devem ser a mesma quantidade, o resto dos coeficientes será zero. Dessa forma definimos as condições para $q$ que deve ser potência de 2, consequentemente, inteiro e par. garante que $gcd(p,q)=1$. Após definir $n$ podemos definir $q$ a partir da desigualdade;
\begin{equation}
    \frac{q}{8}-2 \leq \frac{2n}{3} \quad\longrightarrow \quad q \leq 8 \bigg(\frac{2n}{3}+2\bigg)
\end{equation}
pode escolher até a maior potência de 2 menor ou igual a esse limite. Caso ainda haja interesse em usar um $q$ maior, a especificação recomenda fixar $d$ em $2 \lfloor n/3\rfloor$. $q$ aumenta o tamanho da chave, impactando o trade-off entre segurança e eficiência.

Definimos os polinômios ternários $\mathcal{T}$ e $\mathcal{T}(d)$, que serão usados tanto na geração das chaves $f$ e $g$, quanto para $r$ e $m$;
\begin{align*}
    \mathcal{T} \quad & \text{com grau } n-2 \text{ coeficientes em } \{-1, 0, 1\}. \\
    \mathcal{T}(d) \quad & \text{com grau } n-2 \text{ coeficientes em } \{-1, 0, 1\}. \\
\end{align*}

De acordo com as específicações do hps2048509 adotamos $p=3$, e $n=509$, consequentemente $q=2048$ sendo a maior potência de $2$ abaixo do limite em (1). Portanto $\mathcal{T}(d)$ deve ter 127 coeficientes 1's e 127 coeficientes -1's, restando 254 zeros.

In [500]:
def ternary(n):
    # Z.<xZ> = ZZ[]
    return  Z(random.choices([-1, 0, 1], k=n-1))

def ternary_fixed(n,q):
    d = q//8-2
    ind = list(range(n-1))
    random.shuffle(ind)
    pos = ind[0:d//2]
    neg = ind[d//2:d]
    c = [0]*(n-1)

    for i in pos:
        c[i] = 1
    for i in neg:
        c[i] = -1
    # Z.<xZ> = ZZ[]
    return Z(c)

In [501]:
def count(poly):
    coeffs = poly.list()
    zeros = coeffs.count(0)
    ones = coeffs.count(1)
    neg_ones = coeffs.count(-1)
    return zeros, ones, neg_ones

In [502]:
g = ternary_fixed(n, q)
print("g: ", g)
print("g_in_S3", canonicalNTRU(S3, g.list(), n, q))

zeros, ones, neg_ones = count(g)

print(f"Zeros: {zeros}")
print(f"Uns: {ones}")
print(f"Menos Uns: {neg_ones}")

g:  xZ^506 - xZ^505 + xZ^503 + xZ^502 + xZ^501 - xZ^500 - xZ^499 - xZ^494 - xZ^491 - xZ^490 - xZ^489 - xZ^488 + xZ^487 - xZ^485 - xZ^482 - xZ^478 + xZ^475 + xZ^469 - xZ^468 - xZ^466 - xZ^465 + xZ^464 + xZ^462 + xZ^461 - xZ^460 + xZ^457 - xZ^456 + xZ^452 + xZ^451 + xZ^449 + xZ^448 + xZ^445 - xZ^442 + xZ^440 - xZ^439 - xZ^438 - xZ^433 + xZ^431 - xZ^428 + xZ^427 - xZ^425 - xZ^422 - xZ^419 + xZ^415 - xZ^414 + xZ^411 - xZ^410 - xZ^408 + xZ^405 + xZ^403 + xZ^401 + xZ^399 - xZ^398 - xZ^396 + xZ^394 + xZ^392 - xZ^390 - xZ^387 - xZ^384 - xZ^383 - xZ^382 - xZ^380 + xZ^377 + xZ^376 + xZ^374 - xZ^373 + xZ^370 - xZ^368 - xZ^367 - xZ^366 - xZ^361 - xZ^359 - xZ^358 - xZ^356 - xZ^353 - xZ^352 + xZ^351 - xZ^347 + xZ^344 - xZ^343 - xZ^342 - xZ^338 + xZ^337 + xZ^335 - xZ^330 + xZ^329 + xZ^328 + xZ^326 + xZ^324 - xZ^323 - xZ^321 - xZ^319 + xZ^317 + xZ^315 + xZ^313 + xZ^312 - xZ^310 + xZ^309 + xZ^305 - xZ^304 - xZ^302 - xZ^300 + xZ^296 + xZ^292 + xZ^291 + xZ^287 - xZ^284 - xZ^272 + xZ^262 - xZ^259 - xZ^258

#### Inversa

O artigo não específica o método adotado para obter a inversa, neste caso é utilizado a técnica Hensel lifiting

In [503]:
def exponentiation(base, exp):
    result = 1
    current_power = base
    current_exponent = exp

    while current_exponent > 0:
        if current_exponent % 2 == 1:
            result *= current_power
        current_power *= current_power
        current_exponent //= 2
    return result

# def S2_inverse(S2, a):
#     # aa = S2(a.lift())^(2**508 - 2)
#     aa = exponentiation(S2(a.list()), 2**507 - 1)
#     aa = aa^2
#     return aa

def S2_inverse(S2, a):
    # aa = S2(a.lift())^(2**508 - 2)
    aa = exponentiation(S2(a.lift()), 2**507 - 1)
    aa = aa^2
    return aa

In [504]:
# def Sq_inverse(S2, Sq, a):
#     # v0 = S2(a.lift())^(-1)
#     v0 = S2_inverse(S2, a)

#     # v0 = Sq(v0.lift())
#     # v0 = canonicalNTRU(Sq, v0.list(), q, n)
    
#     for i in range(4):
#         v0 = canonicalNTRU(Sq, v0.list(), q, n) # para n ter erro de coerção entre a * v0
#         v0 = Sq(v0 * (2 - a * v0))
#     return v0

def Sq_inverse(S2, Sq, a):
    # v0 = S2(a.lift())^(-1)
    v0 = S2_inverse(S2, a)
    v0 = Sq(v0.lift())
    for i in range(4):
        v0 = Sq(v0 * (2 - a * v0))
    return v0

## Geração de chaves (KeyGen)

O cálculo da inversa é necessário apenas na etapa de geração de chaves, como mostrado em "DPKE_Public_Key". 

O DPKE_Public_Key chama a função sample_fg para amostrar os polinômios $f$ e $g \in S$
\begin{align*}
    f \quad & \text{com grau } n-2 \text{ coeficientes em } \{-1, 0, 1\}. \\
    g \quad & \text{com grau } n-2 \text{ coeficientes em } \{-1, 0, 1\}. \\
\end{align*}

In [505]:
#section 1.10.1 (pg. 13)
def sample_fg(n, q):
    return ternary(n), ternary_fixed(n,q)

A partir das saidas podemos verificar, nesse caso como p é primo, fp é um corpo de galóis, ou seja sempre haverá inversa nesse anel, permitindo utilizar a função nativa do Sage

In [506]:
def DPKE_Key_Pair(S, S2, S3, Sq, Rq, n, q):
    for attempt in range(5):
        f, g = sample_fg(n, q)
        fp = 1 / canonicalNTRU(S3, f.list(), q, n)
        h, hq, v0, v1, G = DPKE_Public_Key(S, S2, Sq, Rq, n, q, f, g)

        print(f"[n,q]=[{n},{q}], attempt: {attempt + 1}")

        return f, g, fp, h, hq, v0, v1


SageMath representa fielmente o anel S2 das específicações do NTRU. Um dos possíveis erros abaixo, implementação antiga fornece grau n-2, sendo especificado n-1 para Rq

In [507]:
f,g = sample_fg(n,q)
G = 3 * g
v0 = Sq(G * f)
v1 = Sq_inverse(S2, Sq, v0)
G_Sq = Sq(G)
h_Sq = v1 * G_Sq * G_Sq
h1 = Rq(h_Sq.lift()) # modo anterior

G = Z(G.list()) 
v1 = Z(v1.list())

h2 = Rq(v1*G*G) # atual

h2 = canonicalNTRU(Rq, v1.list(), q, n)

# print (G)
# print(v1)
# print(h1)
print(h2)

92*xZ^507 - 48*xZ^506 - 27*xZ^505 + 216*xZ^504 - 170*xZ^503 - 157*xZ^502 + 237*xZ^501 - 187*xZ^500 + 81*xZ^499 - 59*xZ^498 - 53*xZ^497 + 159*xZ^496 + 219*xZ^495 - 18*xZ^494 - 177*xZ^493 + 247*xZ^492 + 171*xZ^491 + 83*xZ^490 + 32*xZ^489 + 57*xZ^488 - 117*xZ^487 - 4*xZ^486 - 149*xZ^485 + 148*xZ^484 + 55*xZ^483 - 248*xZ^482 + 122*xZ^481 + 112*xZ^480 + 93*xZ^479 - 159*xZ^478 - 191*xZ^477 - 100*xZ^476 - 117*xZ^475 + 207*xZ^474 + 79*xZ^473 - 14*xZ^472 + 87*xZ^471 - 194*xZ^470 + 251*xZ^469 + 86*xZ^468 - 160*xZ^467 - 107*xZ^466 - 47*xZ^465 + 201*xZ^464 + 59*xZ^463 - 193*xZ^462 - 35*xZ^461 - 143*xZ^460 + 49*xZ^459 + 101*xZ^458 + 157*xZ^457 - 87*xZ^456 + 30*xZ^455 + 16*xZ^454 + 154*xZ^453 + 160*xZ^452 + 188*xZ^451 + 18*xZ^450 - 135*xZ^449 + 171*xZ^448 + 47*xZ^447 + 72*xZ^446 + 180*xZ^445 - 221*xZ^444 - 238*xZ^443 + 136*xZ^442 + 159*xZ^441 + 145*xZ^440 + 138*xZ^439 - 137*xZ^438 - 133*xZ^437 + 16*xZ^436 - 224*xZ^435 - 56*xZ^434 - 43*xZ^433 + 89*xZ^432 - 234*xZ^431 + 209*xZ^430 + 211*xZ^429 - 58*xZ

In [508]:
def DPKE_Public_Key(S, S2, Sq, Rq, n, q, f, g):
        try:
            G = 3 * g
            v0 = Sq(G * f)
            v1 = Sq_inverse(S2, Sq, v0)

            G = zz(G)
            v1 = zz(v1)
            h = Rq(v1*G*G)
            
            f = zz(f)
            hq = Rq(v1 * f * f)

            return h, hq, v0, v1, G

        except (ZeroDivisionError, ArithmeticError):
            pass

        raise ValueError("limite de tentativas em keygen()")


In [509]:
# def DPKE_Public_Key(S, S2, Sq, Rq, n, q, f, g):
#         try:
#             G = 3 * g
#             v0 = Sq(G * f)
#             v1 = Sq_inverse(S2, Sq, v0)

#             G = zz(G)
#             v1 = zz(v1)
#             h = Rq(v1*G*G)
            
#             f = zz(f)
#             hq = Rq(v1 * f * f)

#             return h, hq, v0, v1, G

#         except (ZeroDivisionError, ArithmeticError):
#             pass

#         raise ValueError("limite de tentativas em keygen()")

In [510]:
f, g, fp, h, hq, v0, v1 = DPKE_Key_Pair(S, S2, S3, Sq, Rq, n, q)

print("f:",  f, "\n")
print("g:",  g, "\n")
print("fp:", fp,"\n")
print("h:",  h, "\n")
print("hq:", hq,"\n")
print("v0:", v0,"\n")
print("v1:", v1,"\n")

[n,q]=[509,2048], attempt: 1
f: -xZ^507 - xZ^505 - xZ^504 - xZ^501 - xZ^500 - xZ^499 - xZ^497 + xZ^496 + xZ^494 - xZ^493 + xZ^492 - xZ^490 - xZ^488 + xZ^487 - xZ^486 + xZ^485 + xZ^482 + xZ^481 - xZ^480 - xZ^479 - xZ^478 + xZ^473 + xZ^470 + xZ^469 + xZ^468 - xZ^467 + xZ^466 - xZ^465 - xZ^464 - xZ^463 - xZ^462 - xZ^461 - xZ^460 - xZ^459 + xZ^458 + xZ^457 - xZ^456 + xZ^455 - xZ^454 - xZ^453 + xZ^449 + xZ^447 + xZ^446 + xZ^445 - xZ^444 + xZ^443 - xZ^442 - xZ^441 + xZ^439 - xZ^438 - xZ^436 + xZ^434 + xZ^433 + xZ^431 + xZ^430 - xZ^427 + xZ^426 - xZ^423 + xZ^422 + xZ^421 - xZ^420 + xZ^419 - xZ^418 - xZ^416 - xZ^415 - xZ^414 + xZ^411 - xZ^410 - xZ^408 + xZ^407 - xZ^406 - xZ^404 - xZ^401 + xZ^399 + xZ^398 - xZ^397 + xZ^396 - xZ^395 - xZ^391 - xZ^389 + xZ^388 - xZ^387 - xZ^386 + xZ^385 + xZ^384 - xZ^383 - xZ^382 + xZ^380 + xZ^376 - xZ^375 - xZ^374 - xZ^373 - xZ^372 - xZ^371 + xZ^370 + xZ^369 - xZ^368 - xZ^366 - xZ^365 - xZ^362 - xZ^361 - xZ^360 - xZ^358 - xZ^357 + xZ^356 + xZ^355 + xZ^354 - xZ^3

## Encrypt

In [511]:
def sample_rm(n, q):
    return ternary(n), ternary_fixed(n,q) 

In [512]:
def DPKE_Encrypt(Rq, h, r, m):
    rq = Rq(r)
    mq = Rq(m)
    c = Rq(((rq*h).lift())+ mq)
    return c

## Decrypt

In [513]:
def decrypt(S3, Sq, Rq, c, f, fp, hq, q):
    v1 = Rq((c*Rq(f)).lift())
    m0 = S3((S3(v1.lift())*S3(fp.lift())).lift()) # fp agora é o polinomio e nao mais anel
    m1 = m0.lift()
    r = Sq((Sq((c-Rq(m1)).lift())*hq).lift())
    r_coeffs = r.lift().list()
    m0_coeffs = m0.lift().list()
    
    r_ok = all(c in {-1, 0, 1} for c in r_coeffs) and (r != 0)
    
    weight = q // 8 - 2
    m0_ok = (all(c in {-1, 0, 1} for c in m0_coeffs) and 
             (m0 != 0) and
             sum(1 for c in m0_coeffs if c == 1) == weight // 2 and
             sum(1 for c in m0_coeffs if c == -1) == weight // 2)
    fail = 0 if (r_ok and m0_ok) else 1
    print(fail)
    return r, m0, fail

## Testes e profiling

In [514]:

class TestNTRU(unittest.TestCase):
    def setUp(self):
        self.qs = {}
        self.rings = {}
        self.keys = {}
        self.qs[509] = 2048
        self.ns = [509]
        self.p = 3
        
        for n in self.ns:
            q = self.qs[n]
            Z.<xZ> = ZZ[]                                   # ZZ[x]
            Z2.<xZ2> = Integers(2)[]                        # (ZZ/2ZZ)[x]
            Z3.<xZ3> = Integers(3)[]                        # (ZZ/3ZZ)[x]
            Zq.<xZq> = Integers(q)[]                        # (ZZ/2048ZZ)[x]
            Phi_n = lambda x : (x^n-1)//(x-1)
            Phi_1 = lambda x : (x - 1)
            R.<xR> = Z.quotient(Phi_1(xZ) * Phi_n(xZ))      # ZZ[x] / (x^n-1)
            S.<xS> = Z.quotient(Phi_n(xZ))                  # ZZ[x] / (x^(n-1) + x^(n-2) + ... + x + 1)
            R3.<xR3> = Z3.quotient(Phi_1(xZ3) * Phi_n(xZ3)) # ZZ[x] / (3, x^n-1)
            Rq.<xRq> = Zq.quotient(Phi_1(xZq) * Phi_n(xZq)) # ZZ[x] / (q, x^n-1)              
            S2.<xS2> = Z2.quotient(Phi_n(xZ2))              # ZZ[x] / (2, x^(n-1) + x^(n-2) + ... + x + 1)
            S3.<xS3> = Z3.quotient(Phi_n(xZ3))              # ZZ[x] / (3, x^(n-1) + x^(n-2) + ... + x + 1)
            Sq.<xSq> = Zq.quotient(Phi_n(xZq))              # ZZ[x] / (q, x^(n-1) + x^(n-2) + ... + x + 1)
            R1.<xR1> = Zq.quotient(Phi_1(xZq))            

            self.rings[(n, q)] = {
                'S'    :  S,
                'Sq'   :  Sq,
                'Rq'   :  Rq,
                'S2'   :  S2,
                'S3'   :  S3,
                'R'    :  R,
                'Phi_n':  Phi_n,
                'R1'   :  R1
            }
            rings = self.rings[(n,q)]

            h, hq, v0, v1, G = DPKE_Public_Key(rings['S'], rings['S2'],
                                    rings['Sq'], rings['Rq'], n, q, f, g)
            
            self.keys[(n,q)] = {
                'f'  : f,
                'g'  : g,
                'fp' : fp,
                'h'  : h,
                'hq' : hq
            }

teste do DPKE_Public_Key

In [515]:

class TestKeygen(TestNTRU):
    def test_parameters(self):
        for n in self.ns:
            q = self.qs[n]
            d = q//8 - 2
            self.assertTrue((q & (q - 1)) == 0)     # q is a power of 2: (2^n = q)
            self.assertGreaterEqual(q, 32)          # q is not small?
            self.assertLessEqual(d, 2*n/3)          # limit of d
            self.assertEqual(gcd(self.p, q), 1)     # p and q are coprime
            self.assertLessEqual(self.p, q)         # p equal or less q 

    def test_ternary(self):
        for n in self.ns:
            q = self.qs[n]
            # rings = self.rings[(n,q)]

            for i in range(1):
                t = ternary(n)

                # t_lift = t.lift()
                self.assertLessEqual(t.degree(), n-2)                      # t of degree at most n-2 (item)              
                self.assertTrue(all(coef in [-1, 0, 1] for coef in t.list()))    # t is ternary polinomial
                self.assertNotEqual(t, 0)                                       # t non-zero ternary polinomial

    def test_ternary_fixed(self):
        for n in self.ns:
            q = self.qs[n]
            # rings = self.rings[(n,q)]

            for i in range(1):
                t_fixed = ternary_fixed(n, q)
                
                # t_fixed_lift = t_fixed.lift()
                self.assertLessEqual(t_fixed.degree(), n-2)                       # t_fixed of degree at most n-2
                self.assertTrue(all(coef in [-1, 0, 1] for coef in t_fixed.list()))        # t_fixed is ternary polynomial
                self.assertNotEqual(t_fixed, 0)                                           # t_fixed non-zero ternary polinomial
                self.assertEqual(sum(1 for coef in t_fixed.list() if coef == 1), q//16-1)  # t_fixed have exactly (q//16-1) coefficients equal to +1
                self.assertEqual(sum(1 for coef in t_fixed.list() if coef == -1), q//16-1) # t_fixed have exactly (q//16-1) coefficients equal to -1           

    def test_sample_fg(self):
        for n in self.ns:
            q = self.qs[n]
            # rings = self.rings[(n, q)]

            for i in range(1):
                f, g = sample_fg(n, q)

                self.assertLessEqual(f.degree(), n-2)                          # f of degree at most n-2 (item)              
                self.assertTrue(all(coef in [-1, 0, 1] for coef in f.list()))        # f is ternary polinomial
                self.assertNotEqual(f, 0)                                           # f non-zero ternary polinomial
                
                self.assertLessEqual(g.degree(), n-2)                          # g of degree at most n-2
                self.assertTrue(all(coef in [-1, 0, 1] for coef in g.list()))        # g is ternary polynomial
                self.assertNotEqual(g, 0)                                           # g non-zero ternary polinomial
                self.assertEqual(sum(1 for coef in g.list() if coef == 1), q//16-1)  # g have exactly (q//16-1) coefficients equal to +1
                self.assertEqual(sum(1 for coef in g.list() if coef == -1), q//16-1) # g have exactly (q//16-1) coefficients equal to -1

    def test_polynomial_ring_keygen(self):
        for n in self.ns:
            q = self.qs[n]
            rings = self.rings[(n, q)]
            keys = self.keys[(n, q)]
            
            G = 3 * keys['g']

            v0 = rings['Sq'](G * keys['f'])
            v1 = Sq_inverse(rings['S2'], rings['Sq'], v0)

            # v0 = (G * keys['f']).lift()
            # v0_sq = canonicalNTRU(rings['Sq'], v0.list(), q, n)

            # v1 = Sq_inverse(rings['S2'], rings['Sq'], v0_sq)
            # v1_sq = canonicalNTRU(rings['Sq'], v1.list(), q, n) 

            self.assertEqual(v0.parent(), rings['Sq'])
            self.assertEqual(keys['h'].parent(), rings['Rq'])
            self.assertEqual(v1.parent(), rings['Sq'])
            self.assertEqual(keys['hq'].parent(), rings['Rq'])

            R1 = rings['R1']
            self.assertEqual(R1(keys['g']), 0) # output section 1.11.2 (notes) 
            
    def test_inverse(self):
        for n in self.ns:
            q = self.qs[n]
            rings = self.rings[(n, q)]
            keys = self.keys[(n, q)]

            for _ in range(1):
                h, hq, v0, v1, G = DPKE_Public_Key(rings['S'], rings['S2'],
                                        rings['Sq'], rings['Rq'], n, q, keys['f'], keys['g'])

                h= zz(h)
                f= zz(keys['f'])
                g= zz(keys['g'])

                f_in_S3 = canonicalNTRU(S3, keys['f'].list(), q, n)

                v0 = zz(v0)
                prod = v0 * v1
                prod = canonicalNTRU(Sq, prod, q, n)
                self.assertEqual(prod, 1)
                
                self.assertEqual(f_in_S3 * fp, 1)

                hq = zz(hq)
                self.assertEqual(canonicalNTRU(Sq, (h * hq).list(), q, n), 1)

                self.assertEqual(Rq(h * f), Rq(3 * g))
                  
                # self.assertEqual(canonicalNTRU(Rq, (h * f).list(), q, n), (3 * keys['g']))

test DPKE_Encrypt

In [516]:
class TestDPKE_Encrypt(TestNTRU):
    def test_sample_rm(self):
        for n in self.ns:
            q = self.qs[n]

            for i in range(1):
                r, m = sample_rm(n, q)

                self.assertLessEqual(r.degree(), n-2)                          # r of degree at most n-2 (item)              
                self.assertTrue(all(coef in [-1, 0, 1] for coef in r.list()))        # r is ternary polinomial
                self.assertNotEqual(r, 0)                                           # r non-zero ternary polinomial
                
                self.assertLessEqual(m.degree(), n-2)                          # m of degree at most n-2
                self.assertTrue(all(coef in [-1, 0, 1] for coef in m.list()))        # m is ternary polynomial
                self.assertNotEqual(m, 0)                                           # m non-zero ternary polinomial
                self.assertEqual(sum(1 for coef in m.list() if coef == 1), q//16-1)  # m have exactly (q//16-1) coefficients equal to +1
                self.assertEqual(sum(1 for coef in m.list() if coef == -1), q//16-1) # m have exactly (q//16-1) coefficients equal to -1
        
    def test_operations(self):
        for n in self.ns:
            q = self.qs[n]
            rings = self.rings[(n,q)]
            keys = self.keys[(n,q)]

            for _ in range(1):
                r, m = sample_fg(n, q)
                c = DPKE_Encrypt(rings['Rq'], keys['h'], r, m)
                self.assertIs(c.parent(), rings['Rq']) # c is an element of Rq

In [517]:
class TestDPKE_Decrypt(TestNTRU):
    def test_sample_rm(self):
        for n in self.ns:
            q = self.qs[n]

            for i in range(1):
                r, m = sample_rm(n, q)

                self.assertLessEqual(r.degree(), n-2)                          # r of degree at most n-2 (item)              
                self.assertTrue(all(coef in [-1, 0, 1] for coef in r.list()))        # r is ternary polinomial
                self.assertNotEqual(r, 0)                                           # r non-zero ternary polinomial
                
                self.assertLessEqual(m.degree(), n-2)                          # m of degree at most n-2
                self.assertTrue(all(coef in [-1, 0, 1] for coef in m.list()))        # m is ternary polynomial
                self.assertNotEqual(m, 0)                                           # m non-zero ternary polinomial
                self.assertEqual(sum(1 for coef in m.list() if coef == 1), q//16-1)  # m have exactly (q//16-1) coefficients equal to +1
                self.assertEqual(sum(1 for coef in m.list() if coef == -1), q//16-1) # m have exactly (q//16-1) coefficients equal to -1

In [518]:
unittest.main(argv=[''], exit=False)
t  = TestKeygen()
t.setUp() 
print("Profiling test_parameters:")
%prun -l 10 t.test_parameters()
t.setUp() 
print("Profiling test_ternary:")
%prun -l 10 t.test_ternary()
t.setUp() 
print("Profiling test_ternary_fixed:")
%prun -l 10 t.test_ternary_fixed() 
print("\nProfiling test_sample_fg:")
t.setUp()
%prun -l 20 t.test_sample_fg() 
print("\nProfiling test_inverse:")
t.setUp()
%prun -l 30 t.test_inverse()

t1 = TestDPKE_Encrypt()
t1.setUp() 
print("Profiling test_operations:")
%prun -l 10 t1.test_operations()

.

........
----------------------------------------------------------------------
Ran 9 tests in 0.223s

OK


Profiling test_parameters:
 Profiling test_ternary:
 Profiling test_ternary_fixed:
 
Profiling test_sample_fg:
 
Profiling test_inverse:
 Profiling test_operations:
 

         2790 function calls in 0.001 seconds

   Ordered by: internal time
   List reduced from 27 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      507    0.000    0.000    0.000    0.000 random.py:242(_randbelow_with_getrandbits)
        1    0.000    0.000    0.001    0.001 1188586069.py:5(ternary_fixed)
        1    0.000    0.000    0.000    0.000 random.py:350(shuffle)
        1    0.000    0.000    0.000    0.000 random.py:454(choices)
        5    0.000    0.000    0.000    0.000 polynomial_quotient_ring_element.py:108(__init__)
        4    0.000    0.000    0.000    0.000 polynomial_ring.py:335(_element_constructor_)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
      693    0.000    0.000    0.000    0.000 {method 'getrandbits' of '_random.Random' objects}
        1    0.000    0.000    0.001    0.001 {built-in method builtins.exec}
      507    0.000    0.000    0.00